In [4]:
import nltk
import pandas as pd
import string
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
from collections import Counter

In [ ]:
books = pd.read_pickle("bigger_romance.pkl")

In [ ]:
books.head()

In [ ]:
books.shape

In [3]:
stopWords = set(stopwords.words('english'))

In [ ]:
len(stopWords)

##### Description words and ratings

descriptionFiltered=[]
for description in books['description']:
    for word in word_tokenize(description.lower()):
        if blah=dates_reviews[['date_formatted','reviews']].groupby('date_formatted').apply(lambda x: x.sum())word not in stopWords and word not in string.punctuation:
            descriptionFiltered.append(word)

In [ ]:
descriptions=[]
for description in books['description']:
    
    descriptionFiltered=[]
    for word in word_tokenize(description.lower()):
        if word not in stopWords and word not in string.punctuation:
            descriptionFiltered.append(word)
    descriptions.append(descriptionFiltered)

In [ ]:
books.loc[:,'description_bag']=descriptions

In [ ]:
books.head()

In [ ]:
books["description_frequency"]=books["description_bag"].map(lambda x: nltk.FreqDist(x))

In [ ]:
books.head()

In [ ]:
years=books['year']

In [ ]:
years=years.dropna()

In [ ]:
years=years.tolist()

In [ ]:
years=set(years)

In [ ]:
years=list(years)

In [ ]:
type(years)

In [ ]:
years[0]

In [ ]:
my_df=[]
for year in years:
    c=Counter() 
    #year=int(year)
    #print(year)
    for index,row in books.iterrows():
        #print(row['year'])
        if row['year']==year:
            #print(index)
            c= c + row['description_frequency']
        d = {
        'year' : year,
        'freqs': c
    }
    my_df.append(d)

my_df = pd.DataFrame(my_df)

In [ ]:
my_df.head()

In [ ]:
no_years = books[["year"]].groupby(["year"]).size()

In [ ]:
no_years.head()

In [ ]:
no_years.reset_index()

In [ ]:
counts=no_years.tolist()

In [ ]:
type(no_years)

In [ ]:
my_df['count']=counts

In [ ]:
my_df.head()

In [ ]:
my_df["freqs"].map(lambda x: x.get("cowboy"))

### Descriptions with scores

In [ ]:
tfDescriptions = TfidfVectorizer(analyzer='word',min_df=1, 
                                 stop_words = 'english')

In [ ]:
transformed_descriptions= tfDescriptions.fit_transform(books["description"])

In [ ]:
transformed_descriptions

In [ ]:
regrDescriptions = linear_model.LinearRegression()

In [ ]:
regrDescriptions.fit(transformed_descriptions, books["rating"])

In [ ]:
resultDescriptions= pd.DataFrame({"coef":regrDescriptions.coef_,
                        "labels":tfDescriptions.get_feature_names()})

In [ ]:
big_description=[word for description in descriptions for word in description]

In [ ]:
corpus_frequency=nltk.FreqDist(big_description)

In [ ]:
resultDescriptions["count"]=resultDescriptions["labels"].map(lambda x: corpus_frequency.get(x))

In [ ]:
resultDescriptions.tail()

In [ ]:
resultDescriptions.to_pickle("Descriptions_bigger.pkl")

In [ ]:
resultDescriptions.sort_values("coef",inplace=True,ascending=True)

In [ ]:
resultDescriptions[resultDescriptions["count"]>20].tail(100)

Trying to make a list of words for a themes

In [1]:
royalty=['princess','queen','king','royalty','royal','crown', 'castle']
military=['military', 'navy', 'army', 'soldier']
supernatural=['vampire','werewolf','ghost','paranormal','supernatural','psychic','witch']
holiday=['christmas', 'thanksgiving', 'valentine', 'halloween', 'new year']
bdsm=['bdsm','dominant','submissive']
adventure=['quest','warrior','voyage','journey','adventure']
wedding=['wedding','bride','groom']
cowboy=['cowboy','cowgirl','farm']
money=['billionaire', 'rich', 'millionaire', 'wealthy']
traits=['novella','series','debut']
settings=['england','scotland','europe','asia','india','china','japan','hawaii','california','texas','hollywood',
          'ireland','florida','france']

In [2]:
all_key_words=royalty+military+supernatural+holiday+bdsm+adventure+wedding+cowboy+money+traits+settings

In [ ]:
resultDescriptions[resultDescriptions['labels'].isin(holiday)]

In [ ]:
books.head()

##### Description words vs. year

In [ ]:
regrDescriptionYear = linear_model.LinearRegression()

In [ ]:
books.head()

In [ ]:
transformed_descriptions_year=tfDescriptions.transform(books.loc[~(books["year"].isnull()),'description'])

In [ ]:
regrDescriptionYear.fit(transformed_descriptions_year, 
                        books.loc[~(books["year"].isnull()),'year'])

In [ ]:
resultDescriptionYear= pd.DataFrame({"coef":regrDescriptionYear.coef_,"labels":tfDescriptions.get_feature_names()})

In [ ]:
resultDescriptionYear.to_pickle("Description_Year.pkl")

In [ ]:
resultDescriptionYear.sort_values("coef",inplace=True,ascending=True)

In [ ]:
resultDescriptionYear["count"]=resultDescriptionYear["labels"].map(lambda x: corpus_frequency.get(x))

In [ ]:
resultDescriptionYear[resultDescriptionYear["count"]>30]

In [ ]:
resultDescriptionYear[resultDescriptionYear['labels'].isin(military)]

##### Review words vs. ratings 

Redoing review words vs ratings so can pick apart the rating a specific reviewer gave with their specific review

In [ ]:
books.head()

In [ ]:
les_review=books['review_texts'].tolist()

In [ ]:
len(les_review)

In [ ]:
newlist=[]
for i in range(0,11018):
    #print(i)
    x=les_review[i]
    for j in range(0,len(x)):
        #print(j)
        try:
            y=x[j][0]
        except:
            y=''
        newlist.append(y)

In [ ]:
len(newlist)

In [ ]:
#all_reviews=[item for sublist in flat_list for item in sublist]

In [ ]:
all_scores=[score for scores in books['reviewer_scores'] for score in scores]

In [ ]:
len(all_scores)

In [ ]:
scores_reviews=pd.DataFrame(
    {'reviews': newlist,
     'scores': all_scores
    })

In [ ]:
no_blanks=scores_reviews[scores_reviews["scores"]!='']

In [ ]:
no_blanks.shape

In [ ]:
les_scores=no_blanks['scores'].tolist()

In [ ]:
len(les_scores)

In [ ]:
tfReviews = TfidfVectorizer(analyzer='word',min_df=0.001, stop_words = 'english')

In [ ]:
transformed_reviews= tfReviews.fit_transform(no_blanks["reviews"])

In [ ]:
transformed_reviews.shape

In [ ]:
regrReviews = linear_model.LinearRegression()

In [ ]:
regrReviews.fit(transformed_reviews, les_scores)

In [ ]:
resultReviews= pd.DataFrame({"coef":regrReviews.coef_,"labels":tfReviews.get_feature_names()})

In [ ]:
resultReviews.head()

In [ ]:
len(resultReviews)

In [ ]:
resultReviews.to_pickle('bigger_reviews_split.pkl')

In [ ]:
2+2

In [ ]:
blah=resultReviews.sort_values('coef')

In [ ]:
sorted_words=blah['labels'].tolist()

intersting big: consent, raped etc. religious, abusive, pregnant, possesive,

intersting big: shock, journey, survivor, laughed, justice, twists, laugh, brotherhood

In [ ]:
for x in range(4700,4978):
        print(sorted_words[x])

In [ ]:
resultReviews[resultReviews['labels'].isin(money)]

In [ ]:
resultReviews[resultReviews['labels'].isin(military)]

In [ ]:
resultReviews[resultReviews['labels'].isin(holiday)]

In [ ]:
resultReviews[resultReviews['labels'].isin(settings)]

In [ ]:
resultReviews[resultReviews['labels'].isin(supernatural)]

In [ ]:
resultReviews[resultReviews['labels'].isin(adventure)]

In [ ]:
resultReviews[resultReviews['labels'].isin(traits)]

### Reviews individually with their dates

In [ ]:
books.head()

In [ ]:
all_dates=[date for dates in books['review_dates'] for date in dates]

In [ ]:
len(all_dates)

In [ ]:
len(newlist)

In [ ]:
dates_reviews=pd.DataFrame(
    {'reviews': newlist,
     'dates': all_dates
    })

In [ ]:
dates_reviews.head()

In [ ]:
dates_reviews["dts"]=pd.to_datetime(dates_reviews["dates"])

In [ ]:
dates_reviews.head()

In [ ]:
2+2

In [ ]:
dates_reviews['date_formatted'] = dates_reviews['dts'].dt.strftime('%b-%Y')

In [ ]:
dates_reviews.head()

In [ ]:
dates_reviews.to_pickle('bigger_reviews_years.pkl')

In [14]:
dates_reviews = pd.read_pickle("bigger_reviews_years.pkl")

In [15]:
dates_reviews.head()

,dates,reviews,dts,date_formatted
0,"Mar 11, 2010",Opening Line: “The last thing Nina Askew neede...,2010-03-11,Mar-2010
1,"Sep 20, 2009",5 stars – Contemporary Romance40-year-old divo...,2009-09-20,Sep-2009
2,"Feb 27, 2009",Loved it! I have a new favorite author. :-) Th...,2009-02-27,Feb-2009
3,"Dec 10, 2014",⭐️⭐️⭐️⭐️⭐️ 5 STARS⭐️⭐️⭐️⭐️⭐️Jennifer Crusie ca...,2014-12-10,Dec-2014
4,"Jan 14, 2016","With her fairytale marriage-gone-bust, Nina As...",2016-01-14,Jan-2016


In [6]:
blah=dates_reviews[['date_formatted','reviews']].groupby('date_formatted').sum()

In [ ]:
#blah=dates_reviews[['reviews']].groupby('date_formatted').apply(lambda x: x.sum())

In [ ]:
blah.head()

In [7]:
blah.to_pickle('reviews_month_year.pkl')

In [5]:
blah=pd.read_pickle("reviews_month_year.pkl")

In [6]:
review_dates=[]
for review in blah['reviews']:
    
    reviewFiltered=[]
    for word in word_tokenize(review.lower()):
        if word in all_key_words:
            reviewFiltered.append(word)
    review_dates.append(reviewFiltered)

In [ ]:
review_dates=[]
for review in blah['reviews']:
    
    reviewFiltered=[]
    for word in word_tokenize(review.lower()):
        if word not in stopWords and word not in string.punctuation:
            reviewFiltered.append(word)
    review_dates.append(reviewFiltered)

In [1]:
len(review_dates)

NameError: name 'review_dates' is not defined

In [ ]:
len(blah)

In [9]:
blah['review_bag']=review_dates

In [10]:
blah.head()

,reviews,review_bag
date_formatted,,
Apr-2007,Quinton Murphy knows what he likes and what he...,"[rich, series, series, series, england, france..."
Apr-2008,The 4th and final book in the Chesapeake Bay /...,"[series, series, series, wedding, series, para..."
Apr-2009,A great story filled with romance and tenderne...,"[series, california, series, military, series,..."
Apr-2010,It did nothing for me. The aspect of 'small to...,"[millionaire, series, series, series, series, ..."
Apr-2011,Simple Irresistible is the first book in the C...,"[series, series, series, series, wedding, seri..."


In [11]:
blah["review_date_frequency"]=blah["review_bag"].map(lambda x: nltk.FreqDist(x))

In [12]:
blah.head()

,reviews,review_bag,review_date_frequency
date_formatted,,,
Apr-2007,Quinton Murphy knows what he likes and what he...,"[rich, series, series, series, england, france...","{'rich': 1, 'series': 9, 'england': 1, 'france..."
Apr-2008,The 4th and final book in the Chesapeake Bay /...,"[series, series, series, wedding, series, para...","{'series': 105, 'wedding': 15, 'paranormal': 3..."
Apr-2009,A great story filled with romance and tenderne...,"[series, california, series, military, series,...","{'series': 180, 'california': 3, 'military': 3..."
Apr-2010,It did nothing for me. The aspect of 'small to...,"[millionaire, series, series, series, series, ...","{'millionaire': 3, 'series': 258, 'debut': 3, ..."
Apr-2011,Simple Irresistible is the first book in the C...,"[series, series, series, series, wedding, seri...","{'series': 665, 'wedding': 43, 'europe': 9, 'j..."


In [16]:
num_months=dates_reviews [["date_formatted"]].groupby(["date_formatted"]).size()

In [17]:
num_months.reset_index()

,date_formatted,0
0,Apr-2007,28
1,Apr-2008,323
2,Apr-2009,442
3,Apr-2010,752
4,Apr-2011,1911
5,Apr-2012,4304
6,Apr-2013,12676
7,Apr-2014,16866
8,Apr-2015,18228
9,Apr-2016,19759


In [18]:
counts_mo=num_months.tolist()

In [19]:
type(counts_mo)

list

In [20]:
blah['counts']=counts_mo

In [21]:
blah.head()

,reviews,review_bag,review_date_frequency,counts
date_formatted,,,,
Apr-2007,Quinton Murphy knows what he likes and what he...,"[rich, series, series, series, england, france...","{'rich': 1, 'series': 9, 'england': 1, 'france...",28
Apr-2008,The 4th and final book in the Chesapeake Bay /...,"[series, series, series, wedding, series, para...","{'series': 105, 'wedding': 15, 'paranormal': 3...",323
Apr-2009,A great story filled with romance and tenderne...,"[series, california, series, military, series,...","{'series': 180, 'california': 3, 'military': 3...",442
Apr-2010,It did nothing for me. The aspect of 'small to...,"[millionaire, series, series, series, series, ...","{'millionaire': 3, 'series': 258, 'debut': 3, ...",752
Apr-2011,Simple Irresistible is the first book in the C...,"[series, series, series, series, wedding, seri...","{'series': 665, 'wedding': 43, 'europe': 9, 'j...",1911


In [23]:
blah.to_pickle("review_word_frequency_by_month_bigger.pkl")

In [ ]:
blah["review_date_frequency"].map(lambda x: x.get("cowboy"))

Old way (all smushed together)

In [ ]:
books["reviews_together"]= books["reviews"].map(lambda x:" ".join(x))  

In [ ]:
tfReviews = TfidfVectorizer(analyzer='word',min_df=1, stop_words = 'english')

In [ ]:
transformed_reviews= tfReviews.fit_transform(books["reviews_together"])

In [ ]:
transformed_reviews

In [ ]:
regrReviews = linear_model.LinearRegression()

In [ ]:
regrReviews.fit(transformed_reviews, books["rating"])

In [ ]:
resultReviews= pd.DataFrame({"coef":regrReviews.coef_,"labels":tfReviews.get_feature_names()})

In [ ]:
resultReviews.head()

In [ ]:
resultReviews["count"]=resultReviews["labels"].map(lambda x: corpus_frequency.get(x))

In [ ]:
resultReviews.to_pickle("Reviews_ratings.pkl")

In [ ]:
resultReviews.sort_values("coef",inplace=True,ascending=True)

In [ ]:
resultReviews.head()

In [ ]:
resultReviews[resultReviews["count"]>10]

In [ ]:
sortthis = resultReviews.sort_values("count",ascending=False)

In [ ]:
sortthis.head(50)